In [0]:
# Gold from Silver (SQL-only, run in PySpark notebook)

# Silver tables
SILVER_CLIENT = "silver_client"
SILVER_PRODUCT = "silver_product"
SILVER_CHANNEL = "silver_channel"
SILVER_MKT_PREF = "silver_consent" #SCD2

# 3) C360_CLIENT_PRODUCTS
# logic：from silver_product_event find profucts held by client（Approved）
spark.sql(f"""
CREATE OR REPLACE TABLE c360_client_products AS
WITH latest_per_product AS (
SELECT *
FROM (
SELECT
cast(client_id as bigint) AS client_id,
cast(product_code as string) AS product_code,
cast(product_event_type as string) AS product_event_type,
cast(product_event_date as date) AS product_event_date,
row_number() OVER (
PARTITION BY client_id, product_code
ORDER BY product_event_date DESC NULLS LAST
) AS rn
FROM {SILVER_PRODUCT}
) x
WHERE rn = 1
),
holdings AS (
SELECT
client_id,
sort_array(collect_set(product_code)) AS product_holdings
FROM latest_per_product
WHERE product_event_type = 'Approved'
GROUP BY client_id
)
SELECT
a.client_id,
b.CLIENT_SERVICE_START_DATE AS start_date,
date'9999-12-31' AS end_date,
1 AS current_ind,
a.product_holdings,
current_date() AS proc_dt
FROM holdings a left join {SILVER_CLIENT} b 
on a.client_id=b.client_id

""")

spark.sql(f"select * from c360_client_products").display()